In [1]:
%pip install boto3==1.34.93 ipython --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import json, re
%time
session = boto3.session.Session()
bedrock_client = session.client(
    service_name='bedrock-runtime',
    region_name='eu-west-3',
)

# Model id and Claude config
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
claude_config = {
    'max_tokens': 4096,
    'temperature': 0, 
    'anthropic_version': '',
    'stop_sequences': ['Human:']
}

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


In [3]:
# Fill with your json sample files here
cdc_json_file_path = "/workspace/formatted_json_cdc1.json" 
cv_json_file_path = "/workspace/formatted_json_cv1.json"
rec_json_file_path = "/workspace/formatted_json_rec1.json"

# Open the JSON file and load its contents
with open(cdc_json_file_path, "r") as file:
    cdc_json_data = json.load(file)
with open(cv_json_file_path, "r") as file:
    cv_json_data = json.load(file)
with open(rec_json_file_path, "r") as file:
    rec_json_data = json.load(file)

# Now you can work with the loaded JSON data
print(cdc_json_data)
print(cv_json_data)

# Add you json inputs here
REQUIREMENTS = cdc_json_data
RESUME = cv_json_data

{'objectif_de_la_mission': ["La mission vise à décharger les équipes SNCF dans trois domaines concernant les services bureautiques et digitaux : la gestion courante des sollicitations et demandes, l'assistance et le conseil aux utilisateurs, et la gestion des stocks.", 1], 'duree_de_la_mission': [210, 3], 'nombre_de_profils': [1, 2], 'types_de_profils': [{'type': ['701.E.PAR - Support Méthode, qualité et Sécurité Expert (ouvert à des profils Senior 701.S.PAR)', 2]}], 'taches_et_responsabilites': [{'tache_et_responsabilite': ['Traiter les demandes des agents G&C reçues via le portail interne Digital Office', 2]}, {'tache_et_responsabilite': ["Prendre en charge les déclarations d'incidents ou d'anomalie de fonctionnement bureautique", 2]}, {'tache_et_responsabilite': ['Alerter / escalader en interne DDT et/ou chez les fournisseurs', 2]}, {'tache_et_responsabilite': ['Assurer le lien avec les différentes assistances et support', 2]}, {'tache_et_responsabilite': ["Gérer le stock, les dotat

In [4]:
CRITERIAS  = rec_json_data['sous_criteres_techniques_et_ponderation']

for critere_json in rec_json_data['sous_criteres_techniques_et_ponderation']:
    print(critere_json['critere'][0])

Qualité de la réponse (Compréhension du besoin, le cas échéant outils et méthodologie proposés)
Nombre et pertinence des expériences similaires
Compétences techniques
Connaissance du domaine fonctionnel
Expériences similaires en accompagnement au changement, actions pédagogiques illustrant la capacité d'écoute, de reformulation et de synthèse


In [5]:
SCALE = {
    0: "Ne répond pas au besoin",
    1: "Ne répond pas suffisamment au besoin",
    2: "Répond partiellement au besoin",
    3: "Répond correctement au besoin",
    4: "Très bonne réponse",
    5: "Réponse au-delà des attentes"
}


In [6]:
prompt = f"""
Vous êtes un assistant d'analyse des compétences techniques de CV dans le cadre de réponse à appel
d'offre du groupe SNCF. Votre tâche est d'évaluer un CV par rapport à un ensemble de critères
techniques pondérés fournis.

Voici les informations dont vous disposez :

<Requirements>
{REQUIREMENTS}
</Requirements>

<Criteria>
{CRITERIAS}
</Criteria>

<Resume>
{RESUME}
</Resume>

<Scale>
{SCALE}
</Scale>
""" + """
Commencez par lire attentivement les exigences et les critères. Ensuite, analysez le CV par rapport
à chaque critère. Pour chaque critère, fournissez d'abord une justification détaillée de la note que
vous attribuez au candidat. Utilisez les informations du CV pour expliquer pourquoi vous pensez que
le candidat mérite cette note pour ce critère spécifique.

Après avoir fourni la justification, donnez une note numérique entre 1 et 5 en vous basant sur
l'échelle de notation fournie.

Une fois que vous avez analysé tous les critères, produisez un JSON avec les notes et les
justifications pour chaque critère.

Le format du JSON doit être le suivant :

{
'critère 1': {
'justification': 'Justification détaillée pour la note',
'score': X
},
'critère 2': {
'justification': 'Justification détaillée pour la note',
'score': Y
},
...
}

Remplacez "critère 1", "critère 2", etc. par les noms réels des critères. Assurez-vous que les
justifications sont claires et détaillées, et que les scores sont conformes à l'échelle de notation
fournie.

<scratchpad>
Vous pouvez utiliser cette zone pour prendre des notes et organiser vos pensées avant de produire le
JSON final.
</scratchpad>

<json_output>
Écrivez votre sortie JSON ici.
</json_output>
"""

In [7]:
def prepare_bedrock_request(messages, claude_config):
    return {
        'messages': messages,
        **claude_config,
    }
def extract_bedrock_response(response):
    body = json.loads(response['body'].read().decode('utf-8'))
    return body['content'][0]['text']

def pretty_print(message):
    print('\n\n'.join('\n'.join(line.strip() for line in re.findall(r'.{1,100}(?:\s+|$)', paragraph.strip('\n'))) for paragraph in re.split(r'\n\n+', message)))

In [8]:
%%time
messages = [
    {"role": "user", "content": [
        {"type": "text", "text": prompt},
        ]
    }
]

body = prepare_bedrock_request(messages, claude_config)
response = bedrock_client.invoke_model(modelId=model_id, body=json.dumps(body))
formated_response = extract_bedrock_response(response)

CPU times: user 31.1 ms, sys: 0 ns, total: 31.1 ms
Wall time: 25.9 s


In [9]:
pretty_print(formated_response)

Voici mon analyse détaillée du CV par rapport aux critères fournis :

<scratchpad>
Critère 1 : Qualité de la réponse (Compréhension du besoin, le cas échéant outils et méthodologie
proposés)
- Le CV ne fournit pas d'informations spécifiques sur la compréhension du besoin ou les
outils/méthodologies proposés pour cette mission. Il s'agit principalement d'un récapitulatif des
expériences professionnelles passées.
=> Score : 2 (Répond partiellement au besoin)

Critère 2 : Nombre et pertinence des expériences similaires
- Le candidat a 2 ans et 6 mois d'expérience en tant que technicien d'assistance et maintenance chez
la SNCF, travaillant sur Windows, Mac OS, Office 365, Active Directory et VPN, ce qui est très
pertinent pour cette mission.
- Il a également 3 ans d'expérience en tant qu'ingénieur support chez Capgemini, travaillant sur des
technologies similaires comme Windows Server et Microsoft Exchange.
- Son expérience chez HELPLINE en tant qu'ingénieur d'affaires semble également per